In [ ]:
from transformers import DistilBertForMaskedLM, DistilBertTokenizerFast
from transformers import PretrainedConfig
from tqdm import tqdm
import numpy as np

%load_ext autoreload
%autoreload 2

model = DistilBertForMaskedLM.from_pretrained("./model_dbertft/", from_tf=True)

tokenizer = DistilBertTokenizerFast.from_pretrained("bert-base-uncased")

In [ ]:
# text = "The capital of France, {} , contains the Eiffel Tower, a very popular tourist attraction."

# vector_input = tokenizer.encode(text, return_tensors="pt")

# probabilities = model(vector_input)[0]

In [ ]:
from utils import get_news_list, get_users_list

news = get_news_list()
users = get_users_list()

users_dict = {}
for user in users:
    users_dict[user.user_id] = user

In [ ]:
from models import get_cosine_similarity, get_user_embeddings, get_news_embeddings

In [ ]:
historical_embeddings, current_embeddings = get_user_embeddings(news[:30], news[0])

get_cosine_similarity(historical_embeddings[0], current_embeddings[0])

In [ ]:
#nDCG25

In [ ]:
import joblib

test_batches = joblib.load("test_batches.pkl")

In [ ]:
for user in test_batches.keys():
    print(users_dict[user].train_list)
    break

In [ ]:
import joblib

final_results = {}
with tqdm(total=len(test_batches)) as pbar:
    for user_key, batch in test_batches.items():
        pbar.update(1)
        test_batch_news_text, test_batch_isread, test_batch_key = batch["news_texts"] , batch["news_is_relevant"], batch["key"]
        historical_embeddings = None
        scores = []
        historical_news = users_dict[user_key].train_list[:5]
        for news in test_batch_news_text:
            current_news = news
            historical_embeddings, current_embeddings = get_user_embeddings(historical_news, current_news, historical_embeddings)
            scores.append(get_cosine_similarity(historical_embeddings[0], current_embeddings[0]))
        scores = np.array(scores)
        final_results[user_key] = [(key, score, is_relevant) for key, score, is_relevant in zip(test_batch_key, scores, test_batch_isread)]
        joblib.dump(final_results, "ranking_dbertft.pkl")


In [ ]:
from sklearn.metrics import ndcg_score

ndcg_score(is_relevant.reshape(1, -1), scores.reshape(1, -1))